In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoMLP
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelMLPProcessor:
    def __init__(self, dates=[]):
        self.dates = dates
        self.forecasts = []
        self.eval_pairs = []
        self.input_size = None
        self.nf = None
        self.config = None
        self.testset = None
        self.maes = []
        self.mses = []
        self.mapes = []
        self.nmses = []
        self.reference_dates = {}
        self.metrics_df = pd.DataFrame(columns=["Unique_id","Reference Date", "MAE", "MSE", "MAPE", "NMSE"])
        self.display_df = pd.DataFrame(columns=["Unique_id","Reference Date", "Target End Date", "GT" , "Quantile", "Prediction"])
        self.model = None
        self.selected_input_size = None
        self.unique_ids = []

    def create_fixed_model(self, df_train, df_test, test_ids, h, freq, level=[]):
        input_len  = df_train.groupby("unique_id").size().min()
        max_inp    = max(8, input_len - h - 1)
        self.testset = df_test

        def config(trial):
            return {
                "input_size"   : trial.suggest_int("input_size", 8, max_inp),
                "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"   : trial.suggest_categorical("batch_size", [16, 32, 64]),
                "random_seed"  : trial.suggest_int("random_seed", 1, 99999),
                "accelerator"  : "gpu",
                "devices"      : 2,
                "strategy"     : "auto"
            }

        self.config = config
        if level:
            nf = NeuralForecast(models=[AutoMLP(h=h, backend="optuna",
                                                loss=MQLoss(level=level),
                                                config=self.config)], freq=freq)
        else:
            nf = NeuralForecast(models=[AutoMLP(h=h, backend="optuna",
                                                config=self.config)], freq=freq)

        nf.fit(df=df_train)
        self.nf   = nf
        self.model = nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]
    def sliding_batch_by_index(self, h: int = 4):
        """Mirrors NHITS/N-BEATS batching. Returns merged forecast+truth rows."""
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T  = min(len(d) for d in uid_dates.values())
        s  = self.selected_input_size
        out = []

        for t in tqdm(range(s, T - h + 1), desc="Batched by index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                ref = dates[t-1]; win = dates[t-s:t]
                df_u = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in)
                    refs.append((uid, ref))
            if not batch:
                continue

            fcst_batch = self.nf.predict(df=pd.concat(batch)).set_index("ds")

            for uid, ref in refs:
                f   = fcst_batch[fcst_batch["unique_id"] == uid].reset_index()
                gt  = (self.testset[self.testset["unique_id"] == uid]
                       .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                out.append(merged)

        return pd.concat(out, ignore_index=True)
    
    
    def calculate_metrics(self):
        for forecast_df, truth_df in self.eval_pairs:
            y_true = truth_df.iloc[:, 1]
            y_pred = forecast_df.iloc[:, 0]
            self.maes.append(mean_absolute_error(y_true, y_pred))
            self.mses.append(mean_squared_error(y_true, y_pred))
            self.mapes.append(mean_absolute_percentage_error(y_true, y_pred))
            self.nmses.append(self.mses[-1] / np.var(y_true))

    def create_metrics_df(self):
        for i in range(len(self.dates)):
            self.metrics_df.loc[len(self.metrics_df)] = [
                self.unique_ids[i],
                self.dates[i],
                self.maes[i],
                self.mses[i],
                self.mapes[i],
                self.nmses[i]
            ]

    def create_display_df(self):
        records = []
        testset_indexed = self.testset.set_index(["unique_id", "ds"])

        for i, forecast_df in enumerate(tqdm(self.forecasts, desc="Building display_df")):
            uid = forecast_df["unique_id"].iloc[0]
            reference_date = self.dates[i]

            try:
                gt_series = testset_indexed.loc[uid]["y"]
            except KeyError:
                gt_series = pd.Series()

            for col in forecast_df.columns:
                if col == "unique_id":
                    continue

                if "lo" in col:
                    number = int(col.split("-")[-1])
                    alpha = 1 - (number / 100)
                    quantile = alpha / 2
                elif "hi" in col:
                    number = int(col.split("-")[-1])
                    alpha = 1 - (number / 100)
                    quantile = 1 - (alpha / 2)
                elif col in ["AutoMLP", "AutoMLP-median"]:
                    quantile = 0.5
                else:
                    continue

                df_col = forecast_df[[col]].copy()
                df_col.rename(columns={col: "Prediction"}, inplace=True)
                df_col["Unique_id"] = uid
                df_col["Reference Date"] = reference_date
                df_col["Target End Date"] = df_col.index
                df_col["GT"] = df_col.index.map(gt_series.get)
                df_col["Quantile"] = quantile
                records.append(df_col)

        self.display_df = pd.concat(records)[
            ["Unique_id", "Reference Date", "Target End Date", "GT", "Quantile", "Prediction"]
        ].sort_values(by=["Unique_id", "Reference Date", "Target End Date", "Quantile"]).reset_index(drop=True)

    def efficient_compute_wis(self):
        df = self.display_df.sort_values(by=["Unique_id", "Reference Date", "Target End Date", "Quantile"])
        results = []
        grouped = df.groupby(["Unique_id", "Reference Date", "Target End Date"])

        for (uid, ref_date, tgt_date), group in tqdm(grouped, desc="Computing WIS"):
            gt = group["GT"].iloc[0]
            if 0.5 not in group["Quantile"].values:
                continue
            median_pred = group[group["Quantile"] == 0.5]["Prediction"].iloc[0]
            ae = abs(median_pred - gt)
            wis = ae
            results.append({
                "Unique_id": uid,
                "Reference Date": ref_date,
                "Target End Date": tgt_date,
                "GT": gt,
                "WIS": wis
            })

        return pd.DataFrame(results)

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelMLPProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, test_ids, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 14:33:22,912] A new study created in memory with name: no-name-19e9c333-9926-4f6f-b03a-730286ee0e3a
[rank: 0] Seed set to 2853
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 23     | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [10]:
result=[]

In [11]:
processor.selected_input_size

16

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

Batched by index t:   0%|          | 0/41 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:   2%|▏         | 1/41 [00:37<25:19, 37.99s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:   5%|▍         | 2/41 [01:15<24:27, 37.63s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

Batched by index t:  95%|█████████▌| 39/41 [24:13<01:14, 37.26s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t:  98%|█████████▊| 40/41 [24:49<00:37, 37.12s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

Batched by index t: 100%|██████████| 41/41 [25:26<00:00, 37.23s/it]


In [13]:
all_results

,ds,unique_id,AutoMLP-median,AutoMLP-lo-95,AutoMLP-lo-90,AutoMLP-lo-85,AutoMLP-lo-80,AutoMLP-lo-70,AutoMLP-lo-60,AutoMLP-lo-50,...,AutoMLP-hi-40,AutoMLP-hi-50,AutoMLP-hi-60,AutoMLP-hi-70,AutoMLP-hi-80,AutoMLP-hi-85,AutoMLP-hi-90,AutoMLP-hi-95,y,Reference Date
0,2011-01-08,Y_1,2.816431,-0.313499,0.501687,0.667256,1.425692,1.344420,1.846479,1.951371,...,3.517073,3.742801,4.047240,4.324179,4.685436,4.953253,5.654578,6.417240,6.28019,2011-01-01
1,2011-01-15,Y_1,2.438014,-0.815791,-0.700299,-0.314682,0.348445,1.122785,1.803144,1.943485,...,3.138647,3.548141,3.456279,3.726537,4.444082,5.293510,5.800790,6.947606,3.68098,2011-01-01
2,2011-01-22,Y_1,2.141190,-0.241415,-0.993738,-0.255418,-0.025503,0.796997,1.049733,1.320779,...,2.818907,3.030576,3.290619,3.865720,4.307754,5.047329,6.121437,7.467255,4.43182,2011-01-01
3,2011-01-29,Y_1,2.101996,-0.205075,-0.683451,-0.765024,-0.198994,-0.028937,0.226673,0.524370,...,2.879018,2.957154,3.214025,3.448318,4.368910,4.804975,6.098234,8.071435,8.24176,2011-01-01
4,2021-09-04,Y_1000,49709.441406,22865.205078,26502.392578,34000.113281,35438.609375,32996.582031,39541.191406,43127.828125,...,56019.265625,61699.914062,62415.503906,58422.773438,64528.769531,69074.437500,68459.046875,81034.578125,69372.00000,2021-08-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354235,1955-04-16,Y_9998,0.000914,-0.022308,-0.014290,-0.029498,-0.019265,-0.010113,-0.009031,0.001591,...,0.025095,0.016549,0.025312,0.053453,0.154593,0.271709,0.453193,0.581747,0.00000,1955-03-19
354236,1957-03-02,Y_9999,0.602624,-0.607712,-0.298598,-0.267769,-0.007435,0.002485,0.172169,0.263154,...,0.868326,1.043858,1.264841,1.323786,1.594319,1.817316,2.286555,2.622477,0.00000,1957-02-23
354237,1957-03-09,Y_9999,0.462207,-0.583279,-0.557211,-0.454628,-0.246542,-0.045465,0.075312,0.166882,...,0.808715,0.852639,0.882535,1.116005,1.280545,1.709858,1.973894,2.528091,0.00000,1957-02-23
354238,1957-03-16,Y_9999,0.294386,-0.247932,-0.584846,-0.419609,-0.331488,-0.186779,-0.103576,-0.049259,...,0.574690,0.656519,0.895172,1.054467,1.276418,1.540437,1.914346,2.325977,0.00000,1957-02-23


In [14]:
all_results.to_csv('MLP_concatenated.csv')

In [24]:
display_df = processor.create_display_df()

Building display_df: 100%|██████████| 2160/2160 [01:25<00:00, 25.31it/s]


In [16]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoMLP-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 354240/354240 [03:08<00:00, 1878.36it/s]


In [17]:
pointwise_df.to_csv('MLP_ALL_STATS.csv')

In [25]:
processor.display_df

,Unique_id,Reference Date,Target End Date,GT,Quantile,Prediction
0,Y_1,2011-10-15,2011-10-15,0.0,0.025,-0.149085
1,Y_1,2011-10-15,2011-10-15,0.0,0.050,-0.122528
2,Y_1,2011-10-15,2011-10-15,0.0,0.075,-0.079287
3,Y_1,2011-10-15,2011-10-15,0.0,0.100,-0.045616
4,Y_1,2011-10-15,2011-10-15,0.0,0.150,-0.035602
...,...,...,...,...,...,...
198715,Y_9999,1957-03-02,1957-03-23,0.0,0.850,1.866410
198716,Y_9999,1957-03-02,1957-03-23,0.0,0.900,2.459928
198717,Y_9999,1957-03-02,1957-03-23,0.0,0.925,3.110700
198718,Y_9999,1957-03-02,1957-03-23,0.0,0.950,3.325852


In [26]:
wis_table = processor.efficient_compute_wis()

Computing WIS: 100%|██████████| 8640/8640 [00:03<00:00, 2445.01it/s]


In [27]:
wis_table

,Unique_id,Reference Date,Target End Date,GT,WIS
0,Y_1,2011-10-15,2011-10-15,0.0,0.001453
1,Y_1,2011-10-15,2011-10-22,0.0,0.001038
2,Y_1,2011-10-15,2011-10-29,0.0,0.000124
3,Y_1,2011-10-15,2011-11-05,0.0,0.001128
4,Y_1000,2022-06-11,2022-06-11,0.0,0.001619
...,...,...,...,...,...
8635,Y_9998,1955-03-26,1955-04-16,0.0,0.111284
8636,Y_9999,1957-03-02,1957-03-02,0.0,1.815044
8637,Y_9999,1957-03-02,1957-03-09,0.0,1.431183
8638,Y_9999,1957-03-02,1957-03-16,0.0,0.973335


In [28]:
wis_table.to_csv('WIS_AutoMLP.csv')

In [29]:
mean_wis = np.mean(wis_table['WIS'].values)

In [30]:
mean_wis

4.766354956754725

In [31]:
wis_dfs = [wis_table.iloc[i::4].reset_index(drop=True) for i in range(4)]

In [35]:
np.mean(wis_dfs[3]['WIS'].values)

3.150650023339088